In [2]:
from pathlib import Path
import pdb
import os
import torch
import cv2
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import shutil
import json
%matplotlib inline

In [8]:
from Bezier import bezier_curve
import math

# 参数
save_path = "/home/pantengteng/datasets/bdd100k/seg"
ignore_list = ["road curb", "crosswalk"]
show_img = False #True # True
save_img = False #True #False #  False
GRID_NUM = 100

# 画 点
def draw_point(img, lane_pt, text='', line_color=(255,0,0), text_color=(0, 0, 0)):
    # pdb.set_trace()
    for i in range(len(lane_pt)):
        pt = lane_pt[i]
        # 画圆，圆心为：(160, 160)，半径为：60，颜色为：point_color，实心线
        # cv2.circle(img, (int(pt[0]), int(pt[1])), 3, (0,0,255), 0)
        if text:
            cv2.putText(img, text + "_%d"%i, (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_COMPLEX, 0.5, text_color, 1)
        
        # 画线
        if i < len(lane_pt) - 1:
            next_pt = lane_pt[i + 1]
            cv2.line(img, (int(pt[0]), int(pt[1])), (int(next_pt[0]), int(next_pt[1])), line_color, 8, 4)
            
    
    return img

#计算两线距离
# 只计算起止点，双线的端点顺序可能相反，需要判断
# 需要将点的顺序修正，方便后续平均--->sorted->sort（sorted新建list）
def dist(pset1, pset2):
    set2_down_pt = pset2[-1]
    set2_up_pt = pset2[0]
    set1_down_pt = pset1[-1]
    set1_up_pt = pset1[0]
    
    dis1 = math.sqrt((set1_down_pt[0] - set2_down_pt[0])**2 + (set1_down_pt[1] - set2_down_pt[1])**2)
    dis2 = math.sqrt((set1_up_pt[0] - set2_up_pt[0])**2 + (set1_up_pt[1] - set2_up_pt[1])**2)
    max_dis = max(dis1, dis2)
    return max_dis

# 计算直线距离
def cal_angle(vec1, vec2):
    vec1_norm = math.sqrt(vec1[0]**2 + vec1[1]**2)
    vec2_norm = math.sqrt(vec2[0]**2 + vec2[1]**2)
    vec_dot = vec1[0]*vec2[0] + vec1[1]*vec2[1]
    cos = vec_dot / (vec1_norm * vec2_norm)
    degree = np.arccos(cos) * 180 / np.pi
    return degree

# 融合
# v1:平均：仅适用于点数相同的情况；
# v2:拟合
# def add(pset1, pset2):
#     r_set = []
#     for i in range(len(pset1)):
#         r_set.append([(pset1[i][0] + pset2[i][0]) / 2, (pset1[i][1] + pset2[i][1]) / 2])
#     return r_set

#                 #if lane_pt[0][2] != "C":
#                 #    print("More than 3 points but not Bezier curve!")
#                 #    sp_case += 1
#                 xvals, yvals = bezier_curve(lane_pt, nTimes=1000)
#                 print(len(xvals))
#                 lane_pt = [(xvals[i],yvals[i]) for i in range(len(xvals))]

# def add(obj1, obj2):
#     pset1 = obj1["poly2d"]
#     pset2 = obj2["poly2d"]
#     if pset1[0][2] == "C" or pset1[-1][2] == "C" or pset2[0][2] == "C" or pset2[-1][2] == "C":
#         print("Bezier curve!")
#         xvals1, yvals1 = bezier_curve(pset1, nTimes=100)
#         xvals2, yvals2 = bezier_curve(pset2, nTimes=100)
#         r_set = [[(xvals1[i] + xvals2[i]) / 2, (yvals1[i] + yvals2[i]) / 2] for i in range(len(xvals1))]
#     else:
#         if len(obj1["poly2d"]) == len(obj2["poly2d"]):
#             print("equal num", len(obj1["poly2d"]))
#             r_set = []
#             for i in range(len(pset1)):
#                 r_set.append([(pset1[i][0] + pset2[i][0]) / 2, (pset1[i][1] + pset2[i][1]) / 2])
#     #         if pset1[0][2] == "C":
#     #             print("equal and Bezier curve!")
#     #             xvals, yvals = bezier_curve(r_set, nTimes=1000)
#     #             print(len(xvals))
#     #             r_set = [(xvals[i],yvals[i]) for i in range(len(xvals))]
#             else:
#                 print("Not equal, not Bezier curve!")
#                 r_set = [[(pset1[0][0] + pset2[0][0]) / 2, (pset1[0][1] + pset2[0][1]) / 2],
#                           [(pset1[-1][0] + pset2[-1][0]) / 2, (pset1[-1][1] + pset2[-1][1]) / 2]]
#     return r_set

def add(obj1, obj2):
    pset1 = obj1["poly2d"]
    pset2 = obj2["poly2d"]
#     if pset1[0][2] == "C" or pset1[-1][2] == "C" and pset2[0][2] == "C" or pset2[-1][2] == "C":
#         print("Bezier curve!")
#         xvals1, yvals1 = bezier_curve(pset1, nTimes=100)
#         xvals2, yvals2 = bezier_curve(pset2, nTimes=100)
#         r_set = [[(xvals1[i] + xvals2[i]) / 2, (yvals1[i] + yvals2[i]) / 2] for i in range(len(xvals1))]
#     else:
        
    if len(pset1) == len(pset2):
        # print("equal num", len(obj1["poly2d"]), "curve mode", pset1[0][2])
        r_set = []
        for i in range(len(pset1)):
            r_set.append([(pset1[i][0] + pset2[i][0]) / 2, (pset1[i][1] + pset2[i][1]) / 2])
            
    elif pset1[0][2] == "L" and pset1[-1][2] == "L" and pset2[0][2] == "L" and pset2[-1][2] == "L":
        # print("Not equal, not Bezier curve!")
        r_set = [[(pset1[0][0] + pset2[0][0]) / 2, (pset1[0][1] + pset2[0][1]) / 2],
                  [(pset1[-1][0] + pset2[-1][0]) / 2, (pset1[-1][1] + pset2[-1][1]) / 2]]
    else:
        # 若C则BC化，所以这里只剩一种：即一L，一C
        # print("mix L and C")
        if len(pset1) < len(pset2):
            pset1, pset2 = pset2, pset1
            
        x_arr = np.linspace(pset2[0][0], pset2[-1][0], num=GRID_NUM, endpoint=True)
        y_arr = np.linspace(pset2[0][1], pset2[-1][1], num=GRID_NUM, endpoint=True)

        r_set = []
        for i in range(len(pset1)):
            r_set.append([(pset1[i][0] + x_arr[i]) / 2, (pset1[i][1] + y_arr[i]) / 2])
        
    return r_set

def maybe_same_lane(cur_lane, another_lane):
    if ( cur_lane["category"] == another_lane["category"] and
            cur_lane["attributes"] == another_lane["attributes"] ):
            # cur_lane["id"] == another_lane["id"] + 1 and 
            # dist(cur_lane["poly2d"], another_lane["poly2d"]) < 100 ):
        # print(cur_lane["id"], another_lane["id"])
        return True
    else:
        return False

# 注意：会改变原始的列表
def combine_same_lane(cur_lane, another_lane):
    lane_pt = add(cur_lane, another_lane)
    cur_lane["poly2d"] = lane_pt
    return cur_lane
    
with open(save_path + "/seg_lane.txt", "r") as f:
    all_list = f.readlines()

max_lane_num = 0
total_num = 0
no_lane_num = 0
for img_name in all_list:
    # pdb.set_trace()
    img_name = img_name.strip("\n")
    # img_name = "1e2d42a4-b1b0abcc"#"2ad1aa4b-9852e784"#"3dab6efd-ecb3083b"#"5c60ec08-4fbeeb70"#"1aa6dd36-7ca1967a"#"4c5c24ad-d0338ef4"#"0b2867ef-a6ca437b"#"0cdde16a-e2d9677e" #"4f45d5ce-d87a28f1"
    print(img_name)
    img = cv2.imread(save_path + "/seg_images/" + img_name + ".jpg") #, cv2.IMREAD_GRAYSCALE)
    img1 = img.copy()
    label_file = save_path + "/lane_labels/" + img_name + ".json"
    with open(label_file, encoding="utf-8") as f:
         label_dic = json.load(f)
    
    # 筛选出需要的lane
    same_line_flg = False
    sp_case = 0
    objs = label_dic["frames"][0]["objects"]
    obj_index = 0
    lane_list = []
    while obj_index < len(objs):
        # pdb.set_trace()
        obj = objs[obj_index]
        if "lane" in obj["category"]:
            # 过滤车道线类型
            if (obj["category"][5:] in ignore_list) or (obj["attributes"]["direction"] != "parallel"):
                obj_index += 1
                continue
            # 按照y更新点的顺序，起点在远端
            lane_pt = obj["poly2d"]

            # BC曲线控制点转换为点集
            if len(lane_pt) > 2:
                # pdb.set_trace()
                if lane_pt[0][2] != "C" and lane_pt[-1][2] != "C":
                    # print("More than 3 points but not Bezier curve!")
                    sp_case += 1
                # print("Convert Bezier curve!")
                xvals, yvals = bezier_curve(lane_pt, nTimes=GRID_NUM)
                obj["poly2d"] = [[xvals[i], yvals[i], "C"] for i in range(len(xvals))]

            obj["poly2d"].sort(key=lambda x:x[1])
        
            #计算与水平方向的夹角
            angle = cal_angle((lane_pt[-1][0] - lane_pt[0][0], lane_pt[-1][1] - lane_pt[0][1]), (10, 0))
            if angle < 5 or angle > 175:
                # print("horizontal %d"%obj["id"])
                obj_index += 1
                continue
                
            img1 = draw_point(img1, obj["poly2d"], str(obj["id"]), line_color=(0, 255, 0))
        
            lane_list.append(obj)
            
        obj_index += 1
    
    # 合并双线
    combined_lane_list = []
    lane_index = 0
    while lane_index < len(lane_list):
        left_index = lane_index + 1
        min_distance = 100 # float('inf')
        min_index = lane_index
        # 每条线只能融合一次，最小
        while left_index < len(lane_list):
            cur_lane = lane_list[lane_index]
            another_lane = lane_list[left_index]
            if maybe_same_lane(cur_lane, another_lane):
                # pdb.set_trace()
                cur_distance = dist(cur_lane["poly2d"], another_lane["poly2d"])
                if min_distance > cur_distance:
                    min_distance = cur_distance
                    min_index = left_index
            left_index += 1
        if min_index != lane_index:
            lane_list[lane_index] = combine_same_lane(lane_list[lane_index], lane_list[min_index])
            # print("combine {} and {}".format(lane_list[lane_index]["id"], lane_list[min_index]["id"]))
            lane_list.pop(min_index)

        combined_lane_list.append(lane_list[lane_index])
        lane_index += 1
    
    lane_num = len(combined_lane_list)
    total_num += lane_num
    if lane_num == 0:
        no_lane_num += 1
    else:
        with open(save_path + "/all_with_lane.txt", "a") as f:
            f.write(img_name + "\n")
    print("lane_num", lane_num)
    if max_lane_num < lane_num:
        max_lane_num = lane_num
    
    # Pyten-20201013-SetLaneNumTo4
    # 根据近端y坐标排序
    combined_lane_list = sorted(combined_lane_list, key=lambda x:x["poly2d"][-1][1], reverse=True)
    combined_lane_list = combined_lane_list[:4]
    
    lane_num = len(combined_lane_list)
    print("reduced_lane_num", lane_num)
    
    # 重新排列
    # 依据远端点x坐标进行排序
    combined_lane_list = sorted(combined_lane_list, key=lambda x:x["poly2d"][0][0])
    
    # 可视化
    # 转换label
    label = np.zeros((img.shape[0], img.shape[1]))
    # pdb.set_trace()
    
    for idx,lane in enumerate(combined_lane_list):
        img = draw_point(img, lane["poly2d"], str(idx), line_color=(0,0,255 ))
        label = draw_point(label, lane["poly2d"], line_color=idx+1)
    
# 合并同线，由于情况不多，暂时放弃拟合
#     while lane_index < len(lane_list) - 1:
#         left_index = lane_index + 1
#         while left_index < len(lane_list):
#             cur_lane = lane_list[lane_index]
#             another_lane = lane_list[left_index]
#             if same_lane(cur_lane, another_lane):
#                 lane_list.pop(left_index)
#                 lane_list[lane_index] = combine_same_lane(cur_lane, another_lane)
    
    if save_img:
        # pdb.set_trace()
        # converted_img_10/
        cv2.imwrite(f"{save_path}/convert_test_4lane/{img_name}.jpg", img)
        cv2.imwrite(f"{save_path}/generated_labels_4lane/{img_name}.png", label)
        
    if show_img:
        img = img[:,:,::-1]
        img1 = img1[:,:,::-1]
        plt.figure(figsize=(20,10))
        # plt.imshow(img)
        # plt.imshow(img1)
        # plt.show()
        plt.subplot(1,2,1)
        plt.imshow(img1)
        plt.axis("off")

        plt.subplot(1,2,2)
        plt.imshow(img)
        plt.axis("off")
        plt.show()
print("max_lane_num", max_lane_num)
print("avg_num", total_num/len(all_list))
print("no_lane_num", no_lane_num)
#out_label = np.zeros((img.shape[0],img.shape[1]))
# c5e41dc2-537a34c6

2778a387-cb297866
lane_num 0
reduced_lane_num 0
2f17cfc3-d881a89f
lane_num 0
reduced_lane_num 0
b8781f69-76c72b31
lane_num 5
reduced_lane_num 4
2037003b-35dafee6
lane_num 0
reduced_lane_num 0
7ce0f7a8-8cb3ed4f
lane_num 4
reduced_lane_num 4
478fb6de-e29aadee
lane_num 2
reduced_lane_num 2
a0ea324c-3e4251c3
lane_num 0
reduced_lane_num 0
b6b047b4-3fccc761
lane_num 0
reduced_lane_num 0
6d123c60-f87b97bc
lane_num 5
reduced_lane_num 4
4a396379-37154446
lane_num 0
reduced_lane_num 0
b96d19af-4b6a5a63
lane_num 4
reduced_lane_num 4
38af7ba3-81d74600
lane_num 0
reduced_lane_num 0
2012cb38-e1a58f29
lane_num 0
reduced_lane_num 0
4807682a-a9181e83
lane_num 2
reduced_lane_num 2
0725b3e1-e2d1e6af
lane_num 1
reduced_lane_num 1
c5e41dc2-537a34c6
lane_num 0
reduced_lane_num 0
c8d3c68c-da2089e8
lane_num 3
reduced_lane_num 3
768d0da9-acfee805
lane_num 5
reduced_lane_num 4
2e5ae824-55520bfb
lane_num 1
reduced_lane_num 1
a0c42a1d-16e85492
lane_num 0
reduced_lane_num 0
ab5f4e49-dc086ab3
lane_num 4
reduced_lan

lane_num 2
reduced_lane_num 2
c389631f-a4b2d909
lane_num 4
reduced_lane_num 4
b251064f-e7a165fd
lane_num 2
reduced_lane_num 2
20b3b81c-0ae81a55
lane_num 5
reduced_lane_num 4
07d025f2-86b087d6
lane_num 3
reduced_lane_num 3
70720f4e-c57c2329
lane_num 1
reduced_lane_num 1
927eb28c-78d4a5a5
lane_num 3
reduced_lane_num 3
76273209-14eea8f3
lane_num 0
reduced_lane_num 0
7a3d45f6-7de076b9
lane_num 2
reduced_lane_num 2
0d01bff2-38b969d3
lane_num 0
reduced_lane_num 0
c5b2506d-a03619f7
lane_num 4
reduced_lane_num 4
1d36dcb4-e4b70050
lane_num 0
reduced_lane_num 0
6ab9c6ac-f11b3e40
lane_num 2
reduced_lane_num 2
52457b6d-451c75be
lane_num 4
reduced_lane_num 4
355f56e4-32a8d4e8
lane_num 3
reduced_lane_num 3
1f8ea4b0-73e92d58
lane_num 1
reduced_lane_num 1
65939f30-fc872d46
lane_num 0
reduced_lane_num 0
3f291ac3-53bdc4d2
lane_num 0
reduced_lane_num 0
a4adaa33-3c2c518a
lane_num 2
reduced_lane_num 2
60e3224d-2ede2128
lane_num 3
reduced_lane_num 3
2b9bc4f8-780bd01f
lane_num 0
reduced_lane_num 0
1e2d42a4-b

231e71a1-022c23f6
lane_num 3
reduced_lane_num 3
a92b522c-8a394099
lane_num 3
reduced_lane_num 3
c31aa396-beb4f81b
lane_num 0
reduced_lane_num 0
8626d42d-031e13bf
lane_num 3
reduced_lane_num 3
010fc651-19922861
lane_num 2
reduced_lane_num 2
3781bfbd-5c1e93d2
lane_num 2
reduced_lane_num 2
1608cfe7-32da2c5d
lane_num 3
reduced_lane_num 3
0cf398b3-ce65ab64
lane_num 4
reduced_lane_num 4
b87299bf-d30f284c
lane_num 2
reduced_lane_num 2
9242a263-1253a2b5
lane_num 4
reduced_lane_num 4
3d411c3d-a9202cf2
lane_num 0
reduced_lane_num 0
8b773dfc-355ad21d
lane_num 0
reduced_lane_num 0
3c21d08e-ff3a7666
lane_num 3
reduced_lane_num 3
b9d34460-eb5b8d34
lane_num 4
reduced_lane_num 4
17cc4c34-2d23045f
lane_num 0
reduced_lane_num 0
3bb1095b-e8547171
lane_num 4
reduced_lane_num 4
2d783b3a-a6304dbd
lane_num 0
reduced_lane_num 0
076399f2-4c68835b
lane_num 0
reduced_lane_num 0
b4632db3-9724d92c
lane_num 4
reduced_lane_num 4
6bd2d920-1a672dba
lane_num 4
reduced_lane_num 4
7faea992-779a5bf9
lane_num 2
reduced_lan

lane_num 4
reduced_lane_num 4
598b9219-20698d42
lane_num 0
reduced_lane_num 0
5daa9274-a4a24449
lane_num 1
reduced_lane_num 1
54c3a9ea-9033e7cc
lane_num 3
reduced_lane_num 3
30b82914-0e870b32
lane_num 1
reduced_lane_num 1
00d8944b-e157478b
lane_num 5
reduced_lane_num 4
3f8382ef-6f86f2fa
lane_num 2
reduced_lane_num 2
bd6c7aef-f3373443
lane_num 3
reduced_lane_num 3
11dc1774-6942564c
lane_num 4
reduced_lane_num 4
bb5cc516-235bb167
lane_num 2
reduced_lane_num 2
498daf4e-44bff8cb
lane_num 0
reduced_lane_num 0
1c1de1e7-ba93f9ae
lane_num 5
reduced_lane_num 4
26e7f143-0f785d12
lane_num 0
reduced_lane_num 0
5af52d67-de2ce960
lane_num 5
reduced_lane_num 4
88d7bcfc-2afd992e
lane_num 1
reduced_lane_num 1
082d5dee-53c30dad
lane_num 5
reduced_lane_num 4
ab54b6e9-91a74408
lane_num 1
reduced_lane_num 1
6ba1916a-d52532b4
lane_num 0
reduced_lane_num 0
b74bedfc-9adcb4ef
lane_num 7
reduced_lane_num 4
22d11e91-7f672b92
lane_num 4
reduced_lane_num 4
43331e94-cfdb9952
lane_num 1
reduced_lane_num 1
9573ac6b-8

lane_num 0
reduced_lane_num 0
a9391f8f-83c00a81
lane_num 2
reduced_lane_num 2
c2e3cd3c-27871334
lane_num 6
reduced_lane_num 4
00091078-59817bb0
lane_num 4
reduced_lane_num 4
2e44968d-15c430d5
lane_num 0
reduced_lane_num 0
161386ee-a7326d9c
lane_num 3
reduced_lane_num 3
0e8c2d15-27cabc30
lane_num 2
reduced_lane_num 2
a7e8be68-102d00c2
lane_num 2
reduced_lane_num 2
4012903a-ef0ce742
lane_num 3
reduced_lane_num 3
21da347b-679df204
lane_num 0
reduced_lane_num 0
27c2b3af-e0bdf021
lane_num 4
reduced_lane_num 4
a1f74c74-14cc6e81
lane_num 1
reduced_lane_num 1
58208859-5316743a
lane_num 0
reduced_lane_num 0
6f77d97a-a4cd1bb8
lane_num 2
reduced_lane_num 2
a58f81f2-78ae959f
lane_num 0
reduced_lane_num 0
200407a9-26db4fcd
lane_num 4
reduced_lane_num 4
87bd1118-11c8099a
lane_num 8
reduced_lane_num 4
1061881b-c116b180
lane_num 3
reduced_lane_num 3
14ee6cec-af45a7ed
lane_num 5
reduced_lane_num 4
4d97e3fb-a9cf0e36
lane_num 1
reduced_lane_num 1
ba479cb3-a99a833a
lane_num 0
reduced_lane_num 0
b7bb37d8-c

...

lane_num 1
reduced_lane_num 1
026c7465-d54954fa
lane_num 4
reduced_lane_num 4
2982948f-d628f94b
lane_num 2
reduced_lane_num 2
27f504cb-4d5d1e4c
lane_num 2
reduced_lane_num 2
438a5b14-e58cc72a
lane_num 3
reduced_lane_num 3
c776d10b-ea20be3a
lane_num 0
reduced_lane_num 0
78f5ea10-3b438287
lane_num 4
reduced_lane_num 4
5b9fe74e-3a853597
lane_num 4
reduced_lane_num 4
9e63770c-0a1cdb3a
lane_num 1
reduced_lane_num 1
5cc4b739-85f1811f
lane_num 2
reduced_lane_num 2
958ded86-63bc47a8
lane_num 1
reduced_lane_num 1
b222c329-5dc8dbf7
lane_num 3
reduced_lane_num 3
88b6fbc3-6b892c82
lane_num 2
reduced_lane_num 2
3b6b3e98-f2cd385d
lane_num 2
reduced_lane_num 2
9e9fae53-dd22325d
lane_num 3
reduced_lane_num 3
24770a66-2e9e294f
lane_num 1
reduced_lane_num 1
1ed7cdb8-3eac5527
lane_num 0
reduced_lane_num 0
75cdb8c0-08de301f
lane_num 0
reduced_lane_num 0
0f31361a-7020f98e
lane_num 4
reduced_lane_num 4
c8ed2212-b60fa314
lane_num 6
reduced_lane_num 4
87e5b8c4-7abcf1c9
lane_num 0
reduced_lane_num 0
b8f63f02-f

reduced_lane_num 4
b2be7200-b6f7fe0a
lane_num 0
reduced_lane_num 0
4a344158-d4d37507
lane_num 0
reduced_lane_num 0
c73abec5-609ede44
lane_num 3
reduced_lane_num 3
900ede79-4c1b8ba7
lane_num 5
reduced_lane_num 4
bb6e9fc4-35369ff6
lane_num 6
reduced_lane_num 4
4f062853-eba1b020
lane_num 2
reduced_lane_num 2
a1186e90-f66c3825
lane_num 2
reduced_lane_num 2
1af55d81-20ae3997
lane_num 0
reduced_lane_num 0
40f63744-416f585b
lane_num 2
reduced_lane_num 2
4d93c8f5-41e14195
lane_num 1
reduced_lane_num 1
1f607426-1bc0137f
lane_num 4
reduced_lane_num 4
b4b68779-1c75351a
lane_num 0
reduced_lane_num 0
6cf7a093-0748b3cc
lane_num 5
reduced_lane_num 4
31e12b60-3dfb1965
lane_num 1
reduced_lane_num 1
01c4035b-bcaeb067
lane_num 0
reduced_lane_num 0
2e54b87c-9e02b648
lane_num 1
reduced_lane_num 1
aed501be-3f43bafc
lane_num 1
reduced_lane_num 1
3cda73b9-234b2cf4
lane_num 2
reduced_lane_num 2
c9427dd5-8dd8e51f
lane_num 1
reduced_lane_num 1
c43fe5b8-3d3608ce
lane_num 3
reduced_lane_num 3
a10582c9-b91592a4
lan

lane_num 4
reduced_lane_num 4
b9c52ee6-7709a9c2
lane_num 0
reduced_lane_num 0
217772f3-ffb3b118
lane_num 1
reduced_lane_num 1
25472b9d-1ee06ba8
lane_num 12
reduced_lane_num 4
616a276c-c76f9016
lane_num 0
reduced_lane_num 0
137460d7-d99eba62
lane_num 4
reduced_lane_num 4
7ffcc315-fb980e5a
lane_num 0
reduced_lane_num 0
8daed08e-825f6173
lane_num 4
reduced_lane_num 4
0c1eebb0-dbb11761
lane_num 0
reduced_lane_num 0
172ec6e1-b709d97f
lane_num 5
reduced_lane_num 4
4b04adbd-d62ca67b
lane_num 3
reduced_lane_num 3
2105b3de-6b3f66e8
lane_num 4
reduced_lane_num 4
adfc8f5c-7bea6867
lane_num 2
reduced_lane_num 2
851ec515-15faca1d
lane_num 2
reduced_lane_num 2
0947dc75-ecbc776b
lane_num 0
reduced_lane_num 0
16c2bfc7-49883575
lane_num 2
reduced_lane_num 2
8cb281a4-145328a9
lane_num 0
reduced_lane_num 0
2f22cc24-a30cfb0b
lane_num 2
reduced_lane_num 2
53999d9f-510f446f
lane_num 1
reduced_lane_num 1
c1a07cae-fc54c9dc
lane_num 0
reduced_lane_num 0
80ce1413-656babeb
lane_num 5
reduced_lane_num 4
802ad4e3-

lane_num 1
reduced_lane_num 1
a5feabf5-2927aa63
lane_num 3
reduced_lane_num 3
9e10ef26-3ba7ffe7
lane_num 6
reduced_lane_num 4
5a815a91-ee140052
lane_num 2
reduced_lane_num 2
a822c459-20b67e7f
lane_num 1
reduced_lane_num 1
a8698d5d-c0d1b942
lane_num 6
reduced_lane_num 4
2b92915d-72c85cc0
lane_num 9
reduced_lane_num 4
94bd6fb3-313c940f
lane_num 1
reduced_lane_num 1
4ce22ce5-1bb06ebd
lane_num 0
reduced_lane_num 0
7944f396-d70990af
lane_num 0
reduced_lane_num 0
a248ac9f-5aa31a49
lane_num 1
reduced_lane_num 1
4d9e6fb4-1f3d419f
lane_num 5
reduced_lane_num 4
36d863a3-ca2d5f65
lane_num 3
reduced_lane_num 3
20a8f0f8-2bd78b81
lane_num 3
reduced_lane_num 3
3100e8a0-baaeb277
lane_num 1
reduced_lane_num 1
4fa3cff9-a504c73e
lane_num 0
reduced_lane_num 0
22a14014-5c0db650
lane_num 2
reduced_lane_num 2
ae420565-b8ce7713
lane_num 2
reduced_lane_num 2
4a592fc2-1f10618d
lane_num 1
reduced_lane_num 1
33f572c4-bc2b2915
lane_num 1
reduced_lane_num 1
a775b107-9856bd54
lane_num 4
reduced_lane_num 4
2fdcceaa-4

lane_num 0
reduced_lane_num 0
ad0a8cc7-c3c77b34
lane_num 1
reduced_lane_num 1
3c1a9a7d-c44eeb32
lane_num 0
reduced_lane_num 0
55ebbace-e698654e
lane_num 3
reduced_lane_num 3
78f3bc20-1770c2b4
lane_num 2
reduced_lane_num 2
a38d3f1d-fade0f28
lane_num 6
reduced_lane_num 4
97871f2b-7a4f67ca
lane_num 1
reduced_lane_num 1
827e3a2b-4707bd1f
lane_num 5
reduced_lane_num 4
65a5b3b3-fc48406d
lane_num 0
reduced_lane_num 0
7fe274e7-b2c33776
lane_num 2
reduced_lane_num 2
24bff1fe-5824eac7
lane_num 3
reduced_lane_num 3
a00a8844-949dd76f
lane_num 3
reduced_lane_num 3
488480d6-bc52f7b9
lane_num 0
reduced_lane_num 0
bd354e59-e5a9e8c6
lane_num 5
reduced_lane_num 4
8acb92d1-e08e0ab4
lane_num 3
reduced_lane_num 3
74f2f74a-2c95e01d
lane_num 4
reduced_lane_num 4
2e1e3eaa-4ec66394
lane_num 3
reduced_lane_num 3
9b0e122c-3ead3b61
lane_num 0
reduced_lane_num 0
6d5eb349-439e0c5f
lane_num 3
reduced_lane_num 3
bbb1289a-a3fbe37f
lane_num 6
reduced_lane_num 4
46d07ebf-4aa62be8
lane_num 0
reduced_lane_num 0
07f58d35-3

In [10]:
# 拆分数据集
with open(save_path + "/all_with_lane.txt", "r") as f:
    all_list = f.readlines()
sub_train_list = np.random.choice(all_list, 2000, replace=False)
for data in all_list:
    if data not in sub_train_list:
        with open(save_path + "/new_test.txt", "a") as f:
            f.write(data)
    else:
        with open(save_path + "/new_train.txt", "a") as f:
            f.write(data)

In [30]:
# 可视化label和图片
imgfile = 'image.jpg'
pngfile = 'mask.png'

img = cv2.imread(imgfile, 1)
mask = cv2.imread(pngfile, 0)

contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img, contours, -1, (0, 0, 255), 1)

img = img[:, :, ::-1]
img[..., 2] = np.where(mask == 1, 255, img[..., 2])

plt.imshow(img)
plt.show()

TypeError: 'NoneType' object is not subscriptable

In [3]:
# 测试for的遍历
a = list(range(1,7))
for x in a:
    x += 1
print(a)
for i in range(len(a)):
    a[i] += 1
print(a)

[1, 2, 3, 4, 5, 6]
[2, 3, 4, 5, 6, 7]


In [7]:
# sort 测试 
a = {"1":[[2,3,5],[2,5,4]],"2":[[3,1,7],[4,1,8]]}
print(a)
b = a["1"]
a["1"].sort(key=lambda x:x[2])
# b.sort()
print(a,b)

{'1': [[2, 3, 5], [2, 5, 4]], '2': [[3, 1, 7], [4, 1, 8]]}
{'1': [[2, 5, 4], [2, 3, 5]], '2': [[3, 1, 7], [4, 1, 8]]} [[2, 5, 4], [2, 3, 5]]


TypeError: list indices must be integers or slices, not tuple

(128, 128, 3)

Counter({0: 2714820, 2: 17673, 1: 14985, 5: 10665, 4: 5679, 3: 978})
